# GARLIC demo

General-purpose Adaptive Richardson-Lucy Image Characterisation

# 1. General-purpose

## Import libraries and scripts

In [ ]:
%matplotlib ipympl
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import AutoMinorLocator

import ipywidgets as widgets
from IPython.display import display

import numpy as np
from time import time
from scipy import ndimage, special

import importlib
import scripts
importlib.reload(scripts)

Plotting:

In [ ]:
def new_figure(fig_name, figsize=(10, 5), nrows=1, ncols=1, sharex='col', sharey='row', gridspec_kw={'hspace': 0, 'wspace': 0}):
    plt.close(fig_name)
    fig = plt.figure(fig_name, figsize=figsize)
    axes = fig.subplots(nrows=nrows, ncols=ncols, squeeze=False,
                        sharex=sharex, sharey=sharey,
                        gridspec_kw=gridspec_kw
                       )
    fig.set_tight_layout(True)
    for ax in axes.flat:
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
        ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
        ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
        ax.grid(True, which='both')

    fig.suptitle(fig_name)
    
    return fig, axes

In [ ]:
default_cmap = plt.get_cmap("gist_earth").copy()
default_cmap.set_bad('gray')


def colour_map(ax, cblabel, data, cmap=default_cmap, norm=None, xlabel=None, x=None, ylabel=None, y=None, projection_axis=0):

    if projection_axis is None:
        projection = data
    else:
        #projection = np.nanmean(data, axis=0)
        #projection = np.nanmean((data - np.nanmean(data, axis=0))**3, axis=0)
        projection = np.nanmax(data, axis=0)

    sigmas = np.linspace(-3, 3, 7)
    percentiles = 50 + 50 * special.erf(sigmas / np.sqrt(2))
    ticks = np.nanpercentile(data, percentiles)
    if norm is None:
        if ticks[-1] > 0:
            linthresh = np.median(projection[projection > 0])
            #print(linthresh)
            norm = colors.SymLogNorm(vmin=ticks[0], vmax=ticks[-1], linthresh=linthresh)
        else:
            norm = colors.Normalize(vmin=ticks[0], vmax=ticks[-1])

    if y is None:
        y = np.arange(projection.shape[0])
    if x is None:
        x = np.arange(projection.shape[1])

    im = ax.imshow(projection,
                   extent=(x[0]-(x[1]-x[0])/2, x[-1]+(x[-1]-x[-2])/2, y[0]-(y[1]-y[0])/2, y[-1]+(y[-1]-y[-2])/2),
                   interpolation='nearest', origin='lower',
                   cmap=cmap,
                   norm=norm,
                  )
    ax.set_aspect('auto')
    if xlabel is not None:
        ax.set_xlabel(xlabel)
    if ylabel is not None:
        ax.set_ylabel(ylabel)

    cb = fig.colorbar(im, ax=ax, orientation='vertical', shrink=.9)
    cb.ax.set_ylabel(cblabel)
    if ticks is not None:
        cb.ax.set_yticks(ticks=ticks, labels=[f'{value:.3g} ({percent:.1f}%)' for value, percent in zip(ticks, percentiles)])
    cb.ax.tick_params(labelsize='small')
    
    return im, cb, norm


## Read data

In [ ]:
importlib.reload(scripts.read_data)
object_name, data, true_spectrum = scripts.read_data.run(35, (0, 0, 1))
data_offset = np.nanmin(data)

In [ ]:
data_cmap = default_cmap
data_norm = colors.LogNorm(vmin=np.percentile(data[data>0], 10), vmax=np.percentile(data[data>0], 99))

## Parameters

In [ ]:
residual_accuracy = .01
max_iter = 100
kernel_truncation = 6

In [ ]:
peak_threshold = 1.2
accretion_threshold = .5

# 2. Adaptive Richardson Lucy

Find noise, source, and background scales

In [ ]:
importlib.reload(scripts.diffuse_emission)
noise_scale = scripts.diffuse_emission.find_scale(data)
source_scale = scripts.diffuse_emission.find_scale(ndimage.gaussian_filter(data, noise_scale, truncate=kernel_truncation))
diffuse_scale = scripts.diffuse_emission.find_scale(ndimage.gaussian_filter(data, source_scale, truncate=kernel_truncation))
print(f'Scales: noise = {noise_scale:.2f}, sources = {source_scale:.2f}, diffuse emission = {diffuse_scale:.2f}')

In [ ]:
baseline = np.nanmin(data)
print(f'baseline={baseline:4g}')

In [ ]:
importlib.reload(scripts.multiscale_RL)
smoothing_radii = np.array([noise_scale, source_scale, diffuse_scale]) / np.sqrt(8*np.log(2)) # FWHM -> Gaussian sigma
smoothing_radii = np.array([8, 10, diffuse_scale]) / np.sqrt(8*np.log(2)) # FWHM -> Gaussian sigma
#smoothing_radii = np.array([noise_scale, source_scale])
n_radii = smoothing_radii.size

mRL = scripts.multiscale_RL.run(data - baseline, smoothing_radii)
RL = np.sum(mRL, axis=0)
m_model = np.empty_like(mRL)
for i, radius in enumerate(smoothing_radii):
    m_model[i] = ndimage.gaussian_filter(mRL[i], radius, truncate=kernel_truncation)
model = baseline + np.sum(m_model, axis=0)

# 3. Image characterisation

## Noise

In [ ]:
residual = data - model
noise = np.sqrt(ndimage.gaussian_filter(residual**2, smoothing_radii[0], truncate=kernel_truncation)) #- ndimage.gaussian_filter(residual, diffuse_scale))
mean = np.nanmean(noise)
noise = np.where(np.isfinite(noise), noise, mean)
print(f'noise: {mean:.3g} +- {np.std(noise):.3g} [{np.min(noise):.3g} - {np.max(noise):.3g}]')

## Background

In [ ]:
pixel_type = np.argmax(m_model, axis=0)
compact_source = mRL[0] > mRL[-1]
diffuse_source = mRL[1] > mRL[-1]

In [ ]:
background = np.fmin(m_model[-1], mRL[-1])
background = ndimage.gaussian_filter(background, smoothing_radii[-1])
background = np.nanmedian([m_model[-1], mRL[-1], background], axis=0)
background = ndimage.gaussian_filter(background, smoothing_radii[-1])
background = baseline + smoothing_radii.size*background

In [ ]:
signal = model - background
SN = signal / noise
median_SN = np.nanmedian(SN)

## Tests

In [ ]:
fig_name = 'bg_test'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(12, 6))
axes = fig.subplots(nrows=2, ncols=2, squeeze=False, sharex=True, sharey=True)
fig.suptitle(fig_name)
fig.set_tight_layout(True)

mask = diffuse_source
mask |= compact_source

ax = axes[0, 0]
norm = colour_map(ax, 'data', data)[2]
#ax.contour(np.max(mask.astype(int)*signal/noise, axis=0), levels=[accretion_threshold, peak_threshold], colors=['b', 'b'], linestyles=['--', '-'])
ax.contour(np.max(mask*SN, axis=0), levels=[peak_threshold], colors=['r'], linestyles=['-'])
ax.contour(np.max(true_spectrum/noise, axis=0), levels=[accretion_threshold, peak_threshold], colors=['k', 'k'], linestyles=['--', '-'])

ax = axes[0, 1]
colour_map(ax, 'background', background)

ax = axes[1, 0]
colour_map(ax, 'S/N', signal/noise)
#ax.contour(np.max(mask.astype(int)*signal/noise, axis=0), levels=[accretion_threshold, peak_threshold], colors=['b', 'b'], linestyles=['--', '-'])
ax.contour(np.max(mask*SN, axis=0), levels=[peak_threshold], colors=['w'], linestyles=['-'])
ax.contour(np.max(true_spectrum/noise, axis=0), levels=[accretion_threshold, peak_threshold], colors=['k', 'k'], linestyles=['--', '-'])

ax = axes[1, 1]
colour_map(ax, 'noise', noise)
#colour_map(ax, 'type', -pixel_type)

plt.show()

In [ ]:
index_y = np.random.randint(0, data.shape[1]); index_x = np.random.randint(0, data.shape[2])
#index_y = 132; index_x = 137
#index_y = 128; index_x = 142
#index_y = 30; index_x = 37
index_y = 30; index_x = 43
#index_y = 33; index_x = 139
#index_y = 21; index_x = 143

fig, axes = new_figure('single_spaxel', figsize=(12, 12), nrows=7, sharex='col', sharey=False)

for ax in axes.flat:
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
    ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
    ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
    ax.grid(True, which='both')

vmin, vmax = np.nanpercentile(data, [16, 84])
norm = colors.Normalize(vmin, vmax)
SN_norm = colors.Normalize(vmin=-peak_threshold, vmax=peak_threshold)

ax = axes[0, 0]
#colour_map(ax, 'true', true_spectrum[:, index_y, :].T, projection_axis=None, norm=norm)
colour_map(ax, 'true S/N', (true_spectrum/noise)[:, index_y, :].T, projection_axis=None, cmap='rainbow_r', norm=SN_norm)
ax.axhline(index_x, c='w', ls='--')
ax.set_ylabel('x')

ax = axes[1, 0]
ax.get_shared_y_axes().join(ax, axes[0, 0])
SN_map = (signal/noise)[:, index_y, :].T
colour_map(ax, 'S/N', SN_map, projection_axis=None, cmap='rainbow_r', norm=SN_norm)
ax.axhline(index_x, c='w', ls='--')
mask = diffuse_source[:, index_y, :].T
mask |= compact_source[:, index_y, :].T
mask &= SN_map > accretion_threshold
#ax.contour(mask*SN, levels=[accretion_threshold, peak_threshold], colors=['b', 'b'], linestyles=['--', '-'])#], colors=['w'], alpha=.5)
ax.contour(mask*SN_map, levels=[peak_threshold], colors=['w'], linestyles=['-'])
axes[0, 0].contour(mask*SN_map, levels=[peak_threshold], colors=['w'], linestyles=['-'])
mask &= SN_map > peak_threshold
ax.contour((true_spectrum/noise)[:, index_y, :].T, levels=[accretion_threshold, peak_threshold], colors=['k', 'k'], linestyles=['--', '-'])
ax.set_ylabel('x')

ax = axes[2, 0]
ax.get_shared_y_axes().join(ax, axes[0, 0])
colour_map(ax, 'observed', data[:, index_y, :].T, projection_axis=None, norm=norm)
ax.axhline(index_x, c='w', ls='--')
ax.set_ylabel('x')


ax = axes[3, 0]
ax.plot(data[:, index_y, index_x], 'k-', alpha=.2, label='measured')
ax.plot(true_spectrum[:, index_y, index_x], 'k-', label='true')
ax.plot(model[:, index_y, index_x], 'b-', alpha=.5, label='model')
ax.plot(background[:, index_y, index_x], 'r--', alpha=.5, label='background')
ax.fill_between(np.arange(data.shape[0]),
                (background - noise)[:, index_y, index_x],
                (background + noise)[:, index_y, index_x],
                color='r', alpha=.1,
                label=f'noise: {mean:.3g}$\\pm${np.std(noise):.3g}') # [{np.min(noise):.3g} - {np.max(noise):.3g}]')
#ax.plot((background + peak_threshold*noise)[:, index_y, index_x], 'k--', label=f'peak_threshold={peak_threshold:.2f}')
#ax.plot((background + accretion_threshold*noise)[:, index_y, index_x], 'k:', label=f'accretion={accretion_threshold:.2f}', alpha=.5)
ax.set_ylabel('intensity')
ax.legend()
cb = fig.colorbar(None, ax=ax, orientation='vertical', shrink=.9)



ax = axes[4, 0]
ax.get_shared_y_axes().join(ax, axes[-1, 0])
colour_map(ax, 'observed', data[:, :, index_x].T, projection_axis=None, norm=norm)
ax.axhline(index_y, c='w', ls='--')
ax.set_ylabel('y')

ax = axes[5, 0]
ax.get_shared_y_axes().join(ax, axes[-1, 0])
SN_map = (signal/noise)[:, :, index_x].T
colour_map(ax, 'S/N', SN_map, projection_axis=None, cmap='rainbow_r', norm=SN_norm)
ax.axhline(index_y, c='w', ls='--')
mask = diffuse_source[:, :, index_x].T
mask |= compact_source[:, :, index_x].T
mask &= SN_map > accretion_threshold
#ax.contour(mask*SN_map, levels=[accretion_threshold, peak_threshold], colors=['b', 'b'], linestyles=['--', '-'])
ax.contour(mask*SN_map, levels=[peak_threshold], colors=['w'], linestyles=['-'])
#mask &= SN_map > peak_threshold
ax.contour((true_spectrum/noise)[:, :, index_x].T, levels=[accretion_threshold, peak_threshold], colors=['k', 'k'], linestyles=['--', '-'], alpha=.5)
ax.set_ylabel('y')

ax = axes[6, 0]
#colour_map(ax, 'true', true_spectrum[:, :, index_x].T, projection_axis=None, norm=norm)
colour_map(ax, 'true S/N', (true_spectrum/noise)[:, :, index_x].T, projection_axis=None, cmap='rainbow_r', norm=SN_norm)
#ax.contour(mask*SN_map, levels=[0, peak_threshold], colors=['w', 'w'], linestyles=['--', '-'])
ax.contour(mask*SN_map, levels=[peak_threshold], colors=['w'])
ax.axhline(index_y, c='w', ls='--')
ax.set_ylabel('y')

ax.set_xlabel('channel')

plt.show()

In [ ]:
peak_threshold

# --- OLD

In [ ]:
raise -1

## Background

In [ ]:
pixel_type = np.argmax(m_model, axis=0)
compact_source = mRL[0] > mRL[-1]
diffuse_source = mRL[1] > mRL[-1]

In [ ]:
background = baseline + 3*m_model[-1]
converged = False
while not converged:
    old_bg = background
    background = np.fmin(background, model)
    xx = np.min(background)
    background = scripts.multiscale_RL.run(background-xx, smoothing_radii)[-1]
    background = xx + 3*ndimage.gaussian_filter(background, smoothing_radii[-1])
    change = (background - old_bg) / noise
    a, b = np.min(change), np.max(change)
    print(a, b)
    if a > -1:
        converged = True

In [ ]:
signal = model - background
SN = signal / noise
median_SN = np.nanmedian(SN)

In [ ]:
sorted_by_SN = np.argsort(SN.flat)
sorted_signal_fraction = np.cumsum(signal.flat[sorted_by_SN])
sorted_signal_fraction /= sorted_signal_fraction[-1]
index0 = np.searchsorted(SN.flat[sorted_by_SN], 0)
accretion_threshold = np.interp(0, sorted_signal_fraction[index0:], SN.flat[sorted_by_SN][index0:])

In [ ]:
peak_threshold = max(accretion_threshold, median_SN) + np.sqrt(np.mean((SN[SN < accretion_threshold] - accretion_threshold)**2))

In [ ]:
median_SN

In [ ]:
fig, axes = new_figure('signal-to-noise_thresholds', nrows=2)


ax = axes[0, 0]
ax.set_ylabel('probability density')

ax.hist(SN.flat, bins=np.linspace(SN.flat[sorted_by_SN[0]], 2*peak_threshold - SN.flat[sorted_by_SN[0]], 3*int(1 + np.sqrt(index0))))


ax = axes[1, 0]
ax.set_ylabel('cumulative flux fraction')

ax.plot(SN.flat[sorted_by_SN], sorted_signal_fraction, 'k-')
ax.axvline(accretion_threshold, c='k', ls=':', label=f'accretion_threshold = {accretion_threshold:.2f} $\\sigma$')
ax.axvline(peak_threshold, c='k', ls='--', label=f'peak_threshold = {peak_threshold:.2f} $\\sigma$')

ax.legend()


ax.set_xlabel('signal / noise')
ax.set_xlim(SN.flat[sorted_by_SN[0]], 2*peak_threshold - SN.flat[sorted_by_SN[0]])
plt.show()